In [4]:
!pip install pandas numpy scikit-learn

In [7]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"

DATA_DIR.mkdir(parents=True, exist_ok=True)

print("Base dir:", BASE_DIR)
print("Data dir:", DATA_DIR)


Base dir: C:\Users\newma\Desktop\Crew_Size_and_Hrs_predcition
Data dir: C:\Users\newma\Desktop\Crew_Size_and_Hrs_predcition\data


In [8]:
def simulate_crew_hours_data(n_samples: int=5000,random_state: int=42)->pd.DataFrame:
    rng=np.random.default_rng(random_state)
    # job size and rooms
    job_size_sqft=rng.integers(300,3500,size=n_samples)
    # number of rooms
    num_rooms=np.clip((job_size_sqft/350+rng.normal(0,1,size=n_samples)).round().astype(int),1,8)

    num_heavy_items=np.clip(rng.poisson(lam=num_rooms*0.8),0,30)

    num_light_items=np.clip(rng.poisson(lam=num_rooms*4.0),5,200)

    # distance
    distance_km = rng.gamma(shape=2.0, scale=5.0, size=n_samples)
    distance_km = np.clip(distance_km, 1, 80)

# building info
    floor_number = rng.integers(0, 12, size=n_samples)
    prob_elevator = np.clip(0.1 + floor_number * 0.08, 0, 0.95)
    has_elevator = rng.binomial(1, prob_elevator).astype(bool)

    # latent complexity
    complexity = (
        0.0015 * job_size_sqft
        + 0.6 * num_rooms
        + 0.7 * num_heavy_items
        + 0.1 * num_light_items
        + 0.5 * np.maximum(floor_number - 2, 0)
        + 0.2 * distance_km
        - 2.0 * has_elevator.astype(int)
    )

    # crew size
    raw_crew = 1.5 + 0.015 * complexity + rng.normal(0, 0.7, size=n_samples)
    crew_size_true = np.clip(raw_crew.round(), 2, 8).astype(int)

    # hours
    raw_hours = 1.5 + 0.05 * complexity + rng.normal(0, 1.5, size=n_samples)
    hours_true = np.clip(raw_hours, 2, 16)

    # past averages
    past_avg_hours = np.clip(
        hours_true + rng.normal(0, 1.0, size=n_samples),
        1.5,
        18
    )

    past_avg_crew_size = np.clip(
        crew_size_true + rng.normal(0, 0.5, size=n_samples),
        1,
        10
    )

    df = pd.DataFrame({
        "job_size_sqft": job_size_sqft,
        "num_rooms": num_rooms,
        "num_heavy_items": num_heavy_items,
        "num_light_items": num_light_items,
        "distance_km": distance_km,
        "floor_number": floor_number,
        "has_elevator": has_elevator,
        "past_avg_hours": past_avg_hours,
        "past_avg_crew_size": past_avg_crew_size,
        "crew_size": crew_size_true,
        "hours_required": hours_true,
    })

    return df
    
    
    

In [9]:
# CELL 3: generate and save data

df = simulate_crew_hours_data(n_samples=5000, random_state=42)

print(df.head())
print(df.describe(include="all"))

csv_path = DATA_DIR / "crew_hours_synthetic.csv"
df.to_csv(csv_path, index=False)
print("Saved dataset to:", csv_path)


   job_size_sqft  num_rooms  num_heavy_items  num_light_items  distance_km  \
0            585          1                2                5    10.779912   
1           2776          7                0               23    11.234749   
2           2394          8                6               39     5.536778   
3           1704          6                3               29    10.749851   
4           1685          4                4                9    17.881982   

   floor_number  has_elevator  past_avg_hours  past_avg_crew_size  crew_size  \
0             9          True        3.672054            1.508863          2   
1             5         False        4.972496            2.442126          2   
2             5          True        2.652201            2.849331          2   
3             0         False        2.855227            1.989930          2   
4             7          True        2.929107            1.860670          2   

   hours_required  
0        2.000000  
1        5

In [10]:
# CELL 4: basic preprocessing and train test split

feature_cols = [
    "job_size_sqft",
    "num_rooms",
    "num_heavy_items",
    "num_light_items",
    "distance_km",
    "floor_number",
    "has_elevator",
    "past_avg_hours",
    "past_avg_crew_size",
]

target_cols = ["crew_size", "hours_required"]

X = df[feature_cols].copy()
y = df[target_cols].copy()

X["has_elevator"] = X["has_elevator"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True,
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((4000, 9), (1000, 9), (4000, 2), (1000, 2))